In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from tqdm import tqdm
import matplotlib.pyplot as plt

class RGBAtoRGB(object):
    def __call__(self, image):
        # Convert RGBA image to RGB by removing the alpha channel
        return image.convert("RGB")

# --- 1. Define transformations ---
transform = transforms.Compose([
    RGBAtoRGB(), 
    transforms.ToTensor(),  # Converts [0-255] RGBA to [0.0-1.0], shape [3, H, W]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for RGB
])

# --- 2. Load dataset ---
dataset = datasets.ImageFolder(root='images_original', transform=transform)

image, label = dataset[0]
print(image.shape)  # The transformed image (e.g., torch.Size([3, H, W]))
print(label)  # The class label (e.g., 0 for 'cat')

# Split into train, val, test (80/10/10)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

print(train_ds)

# --- 3. Create DataLoaders ---
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# --- 4. Define CNN Model ---
class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 3 input channels (RGB)
            nn.ReLU(),
            nn.MaxPool2d(2),  # 288x432 -> 144x216
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 144x216 -> 72x108
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 72 * 108, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.classifier(x)
        return x

model = CNNModel(num_classes=10)

# --- 5. Set device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# --- 6. Loss and optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- 7. Training & Validation Loops ---
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0
    pbar = tqdm(loader, desc="Training", unit="batch")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()

        pbar.set_postfix(loss=loss.item(), acc=100.0 * total_correct / ((pbar.n + 1) * loader.batch_size))

    avg_loss = total_loss / len(loader)
    accuracy = total_correct / len(loader.dataset)
    return avg_loss, accuracy

def evaluate(model, loader, criterion, device, tag="Validation"):
    model.eval()
    total_loss, total_correct = 0, 0
    pbar = tqdm(loader, desc=tag, unit="batch")
    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()

            pbar.set_postfix(loss=loss.item(), acc=100.0 * total_correct / ((pbar.n + 1) * loader.batch_size))

    avg_loss = total_loss / len(loader)
    accuracy = total_correct / len(loader.dataset)
    return avg_loss, accuracy

# --- 8. Train the model ---
epochs = 10
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

# --- 9. Test ---
test_loss, test_acc = evaluate(model, test_loader, criterion, device, tag="Testing")
print(f"\nTest Accuracy: {100 * test_acc:.2f}%")


torch.Size([3, 288, 432])
0

Epoch 1/10


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.19batch/s, acc=6.25, loss=2.31]



Epoch 2/10


Validation: 100%|██████████| 4/4 [00:02<00:00,  1.56batch/s, acc=7.03, loss=2.32]



Epoch 3/10


...


Epoch 9/10


Validation: 100%|██████████| 4/4 [00:02<00:00,  1.56batch/s, acc=17.2, loss=1.83]



Epoch 10/10


Testing: 100%|██████████| 4/4 [00:03<00:00,  1.20batch/s, acc=12.5, loss=2.23]


Test Accuracy: 15.84%


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from tqdm import tqdm
import matplotlib.pyplot as plt

class RGBAtoRGB(object):
    def __call__(self, image):
        # Convert RGBA image to RGB by removing the alpha channel
        return image.convert("RGB")

# --- 1. Define transformations ---
transform = transforms.Compose([
    RGBAtoRGB(), 
    transforms.ToTensor(),  # Converts [0-255] RGBA to [0.0-1.0], shape [3, H, W]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for RGB
])

# --- 2. Load dataset ---
dataset = datasets.ImageFolder(root='images_original', transform=transform)

image, label = dataset[0]
print(image.shape)  # The transformed image (e.g., torch.Size([3, H, W]))
print(label)  # The class label (e.g., 0 for 'cat')

# Split into train, val, test (80/10/10)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

print(train_ds)

# --- 3. Create DataLoaders ---
batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# --- 4. Define CNN Model ---
class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 3 input channels (RGB)
            nn.ReLU(),
            nn.MaxPool2d(2),  # 288x432 -> 144x216
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 144x216 -> 72x108
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 72 * 108, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.classifier(x)
        return x

model = CNNModel(num_classes=10)

# --- 5. Set device ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# --- 6. Loss and optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- 7. Training & Validation Loops ---
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0
    pbar = tqdm(loader, desc="Training", unit="batch")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()

        pbar.set_postfix(loss=loss.item(), acc=100.0 * total_correct / ((pbar.n + 1) * loader.batch_size))

    avg_loss = total_loss / len(loader)
    accuracy = total_correct / len(loader.dataset)
    return avg_loss, accuracy

def evaluate(model, loader, criterion, device, tag="Validation"):
    model.eval()
    total_loss, total_correct = 0, 0
    pbar = tqdm(loader, desc=tag, unit="batch")
    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == labels).sum().item()

            pbar.set_postfix(loss=loss.item(), acc=100.0 * total_correct / ((pbar.n + 1) * loader.batch_size))

    avg_loss = total_loss / len(loader)
    accuracy = total_correct / len(loader.dataset)
    return avg_loss, accuracy

# --- 8. Train the model ---
epochs = 150
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    torch.save(model.state_dict(), 'model.pth')
    torch.save(optimizer.state_dict(), 'optimizer.pth')


# --- 9. Test ---
test_loss, test_acc = evaluate(model, test_loader, criterion, device, tag="Testing")
print(f"\nTest Accuracy: {100 * test_acc:.2f}%")


torch.Size([3, 288, 432])
0

Epoch 1/150


Validation: 100%|██████████| 4/4 [00:02<00:00,  1.57batch/s, acc=14.8, loss=2.13]



Epoch 2/150


Validation: 100%|██████████| 4/4 [00:02<00:00,  1.59batch/s, acc=10.9, loss=2.31]



Epoch 3/150


...


Epoch 149/150


Validation: 100%|██████████| 4/4 [00:02<00:00,  1.61batch/s, acc=38.3, loss=0.32]



Epoch 150/150


Testing: 100%|██████████| 4/4 [00:02<00:00,  1.62batch/s, acc=45.3, loss=1.73] 


Test Accuracy: 57.43%
